In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
import sklearn
from sklearn import tree
from sklearn.metrics import accuracy_score

In [2]:
from sklearn.tree import DecisionTreeClassifier

In [3]:
train_df = pd.read_csv("CalHouseTrain.csv", index_col=0)
test_df = pd.read_csv("CalHouseTest.csv", index_col=0)

In [4]:
x_train = train_df.drop('class', axis=1)
y_test = test_df.drop('class', axis=1)

In [5]:
y_train_pred = []
y_test_pred = []

In [6]:
wl = DecisionTreeClassifier(random_state=0, min_impurity_decrease=0.025)
wl.fit(x_train,train_df['class'])
y_train_pred = wl.predict(x_train)
y_test_pred = wl.predict(y_test)

training_accuracy = accuracy_score(y_train_pred,train_df['class'])
test_accuracy = accuracy_score(y_test_pred,test_df['class'])  

In [7]:
test_accuracy

0.612

In [8]:
from sklearn.tree import DecisionTreeClassifier

def bagging_fit(train_df, nb_trees, seeds):
    list_trees = []
    for i in range(nb_trees):
        #Sampling with the same size as training dataset, and allowing replacement
        train_sample = train_df.sample(frac=1, replace= True)
        x_train =train_sample.drop('class', axis = 1)
        y_train =train_sample['class']
        
        #Create the Decision Tree with same parameters of the previous question
        clf = DecisionTreeClassifier(random_state=0, min_impurity_decrease=0.025)
        clf.fit(x_train, y_train)
        
        list_trees.append(clf)
    return list_trees  # TODO: replace by your own python code


In [9]:

def bagging_predict(tree_list, x_test):
    y_predicted = []
    n_observations = x_test.shape[0]
    list_observations = []
    final_prediction = []
    for i in range(n_observations):
        list_observations.append(i)    
    
    frame = pd.DataFrame(columns = list_observations) #We create a empty dataframe with as columns as observations to predict in the test set

    #We add each predicted values from each decision tree to the dataframe, being each prediction a row.
    for i in range(len(tree_list)):
        y_predicted = tree_list[i].predict(x_test)
        frame.loc[i] = y_predicted
    
    #We compute for each column the value that appeared more often. In case of tie, sorted them alphabetically     
    column_names = frame.columns.tolist()
    for i in range(n_observations):
        count_values = frame[column_names[i]].value_counts()
        
        value = count_values.idxmax()
        same_counts = sorted(count_values.index[count_values == count_values[value]])
        most_common_value = same_counts[0]
        final_prediction.append(most_common_value)
        
    return final_prediction  # TODO: replace by your own python code

In [10]:
tree_list = bagging_fit(train_df,100,[])

In [11]:
prediction = bagging_predict(tree_list, y_test)

In [12]:
len(prediction)

500

In [13]:
accuracy_score(prediction, test_df['class'])

0.644

In [17]:
for i in range (10):
    frame3 = pd.DataFrame(columns = ["nb_trees", "TestAcc"])
    for j in range(100):
        tree_list = bagging_fit(train_df,i+1,[])
        prediction = bagging_predict(tree_list, y_test)
        test_acurr= accuracy_score(prediction, test_df['class'])
        
        frame3.loc[j] = [100*(i+1), test_acurr]
    media = frame3['TestAcc'].mean()
    print(media)

0.6168199999999999
0.6077199999999997
0.6362200000000002
0.63258
0.6418200000000002
0.6361800000000002
0.6456599999999998
0.6424600000000001
0.6433600000000003
0.6446000000000006


In [18]:
print(frame3)

    nb_trees  TestAcc
0     1000.0    0.648
1     1000.0    0.648
2     1000.0    0.654
3     1000.0    0.638
4     1000.0    0.620
..       ...      ...
95    1000.0    0.658
96    1000.0    0.646
97    1000.0    0.632
98    1000.0    0.650
99    1000.0    0.660

[100 rows x 2 columns]


In [20]:
for i in range (10):
    frame3 = pd.DataFrame(columns = ["nb_trees", "TestAcc"])
    for j in range(100):
        tree_list = bagging_fit(train_df,(i+1)*5,[])
        prediction = bagging_predict(tree_list, y_test)
        test_acurr= accuracy_score(prediction, test_df['class'])
        
        frame3.loc[j] = [i+1, test_acurr]
    media = frame3['TestAcc'].mean()
    print(media)

0.6433800000000001
0.6445000000000003
0.6473800000000002
0.6479000000000003
0.6474800000000002
0.64656
0.6472800000000002
0.6478800000000001
0.6469600000000004
0.6447800000000005


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to search over
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the Random Forest Classifier
rfc = RandomForestClassifier(random_state=0)

# Perform the grid search
grid_search_rfc = GridSearchCV(rfc, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search_rfc.fit(x_train, train_df['class'])

# Get the best parameters and the corresponding accuracy score
best_params_rfc = grid_search_rfc.best_params_
best_score_rfc = grid_search_rfc.best_score_

print(f'Best parameters for Random Forest Classifier: {best_params_rfc}')
print(f'Best accuracy score for Random Forest Classifier: {best_score_rfc:.3f}')

Best parameters for Random Forest Classifier: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best accuracy score for Random Forest Classifier: 0.766


In [44]:
rfc = RandomForestClassifier(max_depth = None, max_features = 'log2', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 100)

In [45]:
rfc.fit(x_train, train_df['class'])
prediction = rfc.predict(y_test)

In [46]:
accuracy_score(prediction, test_df['class'])

0.722

In [29]:
from sklearn.ensemble import BaggingClassifier

In [35]:
bg = BaggingClassifier(random_state=0, n_estimators = 100)

In [36]:
bg.fit(x_train, train_df['class'])
prediction = rfc.predict(y_test)
accuracy_score(prediction, test_df['class'])

0.726

In [41]:
def bagging_fit(train_df, nb_trees, seeds):
    list_trees = []
    nb_trees = 7 # We have chosen this amount of trees as during the tune of the metaparameters it seems to outperform the rest
    for i in range(nb_trees): 
        #Sampling with the same size as training dataset, and allowing replacement
        train_sample = train_df.sample(frac=1, replace= True, random_state= 0)
        x_train =train_sample.drop('class', axis = 1)
        y_train =train_sample['class']
        
        #Create the Decision Tree with same parameters of the previous question
        clf = DecisionTreeClassifier(random_state=0, min_samples_split = 16, min_impurity_decrease= 0.003)
        clf.fit(x_train, y_train)
        
        list_trees.append(clf)
    return list_trees  # TODO: replace by your own python code

In [43]:
for j in range(100):
    tree_list = bagging_fit(train_df,5,[])
    prediction = bagging_predict(tree_list, y_test)
    test_acurr= accuracy_score(prediction, test_df['class'])
        
    frame3.loc[j] = [100*(i+1), test_acurr]
media = frame3['TestAcc'].mean()
print(media)

0.6379999999999995
